# Try to steer with PC1

In [1]:
import json
import os
import torch
import sys

sys.path.append('.')
sys.path.append('..')

from utils.steering_utils import ActivationSteering
from utils.probing_utils import *
from utils.inference_utils import *

torch.set_float32_matmul_precision('high')

INFO 08-12 23:22:14 [__init__.py:235] Automatically detected platform cuda.


In [2]:
layer = 22

In [3]:
# steering on PC1
rp_basis = torch.load('/root/git/persona-subspace/roles/data/pca_240/layer22_pos23_pc1.pt', weights_only=False)
rp_basis = torch.from_numpy(rp_basis)
print(f"Role PC1 shape: {rp_basis.shape}")

Role PC1 shape: torch.Size([4608])


In [4]:
output_dir = "./results/roles_240/steering"

In [5]:
CHAT_MODEL_NAME = "google/gemma-2-27b-it"
model, tokenizer = load_model(CHAT_MODEL_NAME, device="cuda:0")

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

In [6]:
question_path = '/root/git/persona-subspace/traits/data/questions_240.jsonl'
questions = []
with open(question_path, 'r') as f:
    for line in f:
        questions.append(json.loads(line))


In [7]:
magnitudes = [-10.0, -3.0, -2.0, 1.0, 2.0, 3.0, 10.0]
test_questions = questions[:5]

steered_results = {}

for magnitude in magnitudes:
    print(f"\n{'='*20} Magnitude: {magnitude:+.1f} {'='*20}")
    
    try:
        with ActivationSteering(
            model=model,
            steering_vectors=[rp_basis],
            coefficients=magnitude,
            layer_indices=layer,
            intervention_type="addition",
            positions="all"
        ) as steerer:
            for question in test_questions:
                prompt = question['question']
                if prompt not in steered_results:
                    steered_results[prompt] = {}
                
                print(f"\nPrompt: {prompt}")
                response = generate_text(model, tokenizer, prompt, chat_format=True)
                print(f"Response: {response}")
                if magnitude not in steered_results[prompt]:
                    steered_results[prompt][magnitude] = []
                steered_results[prompt][magnitude].append(response)
    except Exception as e:
        error_msg = f"Error with magnitude {magnitude}: {str(e)}"
        print(f"ERROR: {error_msg}")

with open(f"{output_dir}/pc1.json", "w") as f:
    json.dump(steered_results, f, indent=2)


==================== Magnitude: -10.0 ====================

Prompt: What is the relationship between law and morality?
Response: The relationship between law and morality is complex and multifaceted. There's no single, definitive answer, as different perspectives exist on how they interact.

Here are some key points to consider:

**Overlapping but Distinct:**

* **Law** is a system of rules created and enforced through social or governmental institutions to regulate behavior within a society. It focuses on maintaining order, resolving disputes, and protecting individual rights.

* **Morality**, on the other hand, refers to principles concerning right and wrong conduct, often based on personal beliefs, cultural norms, religion, or philosophy. It guides our actions and judgments about what is good or bad.


While there are overlaps, they aren't synonymous:

* **Overlap:** Many laws are based on moral principles (e.g., laws against murder, theft, assault). Societies tend to codify shared

skipping cudagraphs due to skipping cudagraphs due to cpu device (arg355_1)


Response: The relationship between law and morality is complex and has been debated by philosophers and legal scholars for centuries. There's no single, universally agreed-upon answer, but here are some key perspectives:

**1. Law as a reflection of morality:**

* This view argues that laws should reflect the moral values of society. Laws are seen as codified morality, designed to promote good and prevent harm.

**2. Law as independent of morality:**

* This perspective argues that law and morality are separate domains. Law focuses on maintaining order and resolving disputes, while morality deals with right and wrong. A law can be valid even if it conflicts with individual moral beliefs.

**3. Law influencing morality:**

* Law can shape and influence societal morality over time. By establishing certain behaviors as illegal or unacceptable, the law can nudge people towards adopting those moral norms.

**4. Morality influencing law:**

* Social movements and changing moral attitudes oft